In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wandb
from scripts.models import *
from scripts.utils import custom_metric, Wandb_plot, csv_logger, animate_function
import pandas as pd
import matplotlib.animation as animation
import ast
import networkx as nx
from matplotlib.animation import FuncAnimation, PillowWriter

In [2]:
m = Model(1, [1,4,1], [None, tf.keras.activations.tanh,tf.keras.activations.tanh,tf.keras.activations.tanh,None])
m

2024-08-24 12:27:57.462387: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-24 12:27:57.462421: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-24 12:27:57.462432: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-24 12:27:57.462911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-24 12:27:57.463298: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[layer : layer_0, neurons: 1, layer : layer_1, neurons: 4, layer : layer_2, neurons: 1]

In [3]:
x_train = np.arange(-10, 10, 0.01)
y_train = np.sin(x_train)
y_mean = np.mean(y_train)

In [8]:
# Define learning rate schedule and optimizer
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9)

loss_metric = custom_metric(name = "loss")
accuracy_metric = custom_metric(name = "accuracy")

metrics = {'loss': loss_metric, 'accuracy': accuracy_metric}

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
m.compile(optimizer, metrics, y_mean)
w_callback = Wandb_plot()
logs = csv_logger(name = "test1.csv")
wandb.config.update({"model": m})

In [9]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_train = tf.reshape(x_train, (-1,1))
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_train = tf.reshape(y_train, (-1,1))

In [6]:
m.fit(x_train, y_train, epochs=20, batch_size=10, callbacks=[logs])

Epoch 1/20
  1/200 [..............................] - ETA: 1:07 - loss: 63.7436 - accuracy: -0.0283

2024-08-24 12:28:00.875061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 2s 6ms/step - loss: 6.6066 - accuracy: -161.2495
Epoch 2/20
200/200 [==============================] - 1s 6ms/step - loss: 4.7225 - accuracy: -104.8747
Epoch 3/20
200/200 [==============================] - 1s 6ms/step - loss: 4.6967 - accuracy: -71.5252
Epoch 4/20
200/200 [==============================] - 1s 6ms/step - loss: 4.6792 - accuracy: -242.9244
Epoch 5/20
200/200 [==============================] - 1s 6ms/step - loss: 4.7470 - accuracy: -399.4497
Epoch 6/20
156/200 [======================>.......] - ETA: 0s - loss: 4.7900 - accuracy: -113.2247

KeyboardInterrupt: 

In [7]:
m.predict(x_train).numpy()

array([[-0.3035538 ],
       [-0.30326843],
       [-0.302983  ],
       ...,
       [ 0.26654044],
       [ 0.26682594],
       [ 0.26711133]], dtype=float32)

In [8]:
animate_function(logs= 'logs/test1.csv', name = "test1.gif")

MovieWriter imagemagick unavailable; using Pillow instead.


In [ ]:
m.save("model1")
